In [ ]:
from chess import parse_square
from chess.pgn import Game, FileExporter
from chess.engine import SimpleEngine, Limit
from chess.svg import Arrow

In [ ]:
from datetime import datetime
from tqdm import tqdm

In [ ]:
SF_DIR = 'stockfish_21091218_x64_modern/stockfish_21091218_x64_modern'

# Game opening we'll analyse
BOARD_SETUP = 'rnbqkb1r/ppp1pppp/5n2/3p4/3P1B2/2N5/PPP1PPPP/R2QKBNR b KQkq - 2 3'

# engine settings
MULTIPV = 3
ENGINE_DEPTH = 15
MOVE_DEPTH = 6

In [ ]:
# init
engine = SimpleEngine.popen_uci(SF_DIR)
engine.configure({'Hash': 512, 'Threads':2, 'Use NNUE':True,
                  'EvalFile':'stockfish_21091218_x64_modern/nn-6762d36ad265.nnue'})

limitation = Limit(depth=ENGINE_DEPTH)

game = Game()
game.headers['Event'] = 'Book Generation'
game.headers['Date']  = str(datetime.now())
game.headers['White'] = 'Best Reply'
game.headers['Black'] = 'Opponent'
game.setup(BOARD_SETUP)

In [ ]:
game.board()

In [ ]:
# create function analyse board position
info = lambda board: engine.analyse(board, limitation, multipv=MULTIPV)

In [ ]:
def add_variation_of(board, progress_bar):
   
    # get next (principal) move from board
    for pv in info(board.board()):
        
        is_white_to_move = pv['score'].turn
        pawn_to_move = pv['pv'][0]
        # get score from white PoV
        centipawn_score = pv['score'].white().score()

        # to get minified book, only analyse position where
        # White always play MULTIPV best move:        
        # * now is Black turn
        # * now is White turn with score better than -40
        if (not is_white_to_move) or (centipawn_score > 40):
            board.add_variation(pawn_to_move,
                                comment=str(centipawn_score))
    
    # create arrows showing all added variations
    arrows = []
    for pv in board.variations:
        uci_str = pv.uci()
        arrows.append( Arrow(parse_square(uci_str[:2]),
                             parse_square(uci_str[2:4]))
                     )
    board.set_arrows(arrows)
        
    progress_bar.update(1)

In [ ]:
def _recursive(game, depth, progress_bar):
    
    # stopping condition
    if depth<0: return
    
    for variation in list(game.variations):
        add_variation_of(variation, progress_bar)
        _recursive(variation, depth-1, progress_bar)

In [ ]:
# count maximum variation to analysed
total_variations = (MULTIPV**(MOVE_DEPTH+1)-1) // (MULTIPV-1)
progress_bar = tqdm(total=total_variations)

# create book
add_variation_of(game, progress_bar)
_recursive(game, MOVE_DEPTH-1, progress_bar)

In [ ]:
filename = '{}-PV{}-BD{}-EN{}.pgn'.format('jobava', MULTIPV, MOVE_DEPTH, ENGINE_DEPTH)

with open(filename, "w") as f:
     game.accept(FileExporter(f))